In [1]:
import os
import re
import math
import pandas as pd

In [2]:
Datas = {int(re.findall(r"OpenMP_Thread(\d+).csv", i)[0]): pd.read_csv(f"CannyRecord/Grad/runtime/{i}") for i in os.listdir("CannyRecord/Grad/runtime") if i.endswith(".csv") and i.startswith("OpenMP")}

In [3]:
print(Datas.keys())

dict_keys([9, 6, 7, 14, 15, 11, 1, 3, 5, 10, 16, 8, 2, 4, 12, 13])


In [4]:
for Threads, Data in Datas.items():
    Data['Diagonal Length'] = Data['Width x Height'].apply(
        lambda x: math.sqrt(sum(int(num)**2 for num in x.split('x')))
    )
    Data.sort_values(by=['Diagonal Length', 'Image Name'], ascending=[True, True], inplace=True)

In [6]:
print(Datas)

{9:       Image Name Width x Height  Average Processing Time (ns)  Diagonal Length
0    machine.jpg        640x480                        724163       800.000000
3        dog.jpg        482x640                        696150       801.201598
5        car.jpg       1280x720                       2139896      1468.604780
8        cat.jpg       1280x853                       2565159      1538.183669
6       lake.jpg       1280x855                       2617475      1539.293669
1       cube.jpg      1024x1792                       4095703      2063.937984
13  mountain.jpg      1792x1024                       4042326      2063.937984
14    female.jpg      1920x1080                       4454202      2202.907170
4     street.jpg      1920x1280                       5405064      2307.552816
2      build.jpg      1326x1920                       5530512      2333.382952
11    bakery.jpg      1996x1447                       6259008      2465.324522
9       car2.jpg      2560x1440                 

In [20]:
results = {}

for thread_id, df in Datas.items():
    weighted_times = df['Average Processing Time (ns)'] * df['Diagonal Length']
    total_weighted_time = weighted_times.sum()
    total_diagonal_length = df['Diagonal Length'].sum()
    if total_diagonal_length > 0:
        weighted_average_time = total_weighted_time / total_diagonal_length
    else:
        weighted_average_time = None
    results[thread_id] = weighted_average_time
results = sorted(results.items(), key=lambda item: item[1])
for thread_id, average_time in results:
    print(f"OpenMP_Thread{thread_id} - Weighted Average Processing Time: {average_time}")

OpenMP_Thread16 - Weighted Average Processing Time: 17042848.864888314
OpenMP_Thread15 - Weighted Average Processing Time: 17266203.54682786
OpenMP_Thread14 - Weighted Average Processing Time: 18310323.658959884
OpenMP_Thread13 - Weighted Average Processing Time: 19902084.24767838
OpenMP_Thread12 - Weighted Average Processing Time: 21137266.75593317
OpenMP_Thread7 - Weighted Average Processing Time: 22322350.52728219
OpenMP_Thread8 - Weighted Average Processing Time: 22475483.89701561
OpenMP_Thread11 - Weighted Average Processing Time: 22554414.893255122
OpenMP_Thread10 - Weighted Average Processing Time: 24487171.140380315
OpenMP_Thread9 - Weighted Average Processing Time: 26639504.852140192
OpenMP_Thread6 - Weighted Average Processing Time: 26840046.351362195
OpenMP_Thread5 - Weighted Average Processing Time: 29632753.761920817
OpenMP_Thread4 - Weighted Average Processing Time: 39186797.90306975
OpenMP_Thread3 - Weighted Average Processing Time: 52532234.66356154
OpenMP_Thread2 - Wei

In [7]:
PicDatas = {FileName.split('.')[0]: {'time': {}, 'Len': 0, 'MinIdx': 0} for FileName in Datas[1]['Image Name']}

In [16]:
print(PicDatas['earth'])

{'time': {9: 75238693, 6: 75998426, 7: 60745362, 14: 51616214, 15: 48142123, 11: 63776141, 1: 412423316, 3: 153083840, 5: 84046320, 10: 69181918, 16: 47031003, 8: 54132484, 2: 204002020, 4: 111914724, 12: 59568439, 13: 56699523}, 'Len': 8811.628680329193, 'MinIdx': 16}


In [12]:
for Threads, Data in Datas.items():
    for Index, Row in Data.iterrows():
        Name = Row['Image Name'].split('.')[0]
        PicDatas[Name]['time'][Threads] = Row['Average Processing Time (ns)']
        PicDatas[Name]['Len'] = Row['Diagonal Length']

In [ ]:
MinKeyInMachine = min(PicDatas['machine'], key = PicDatas['machine'].get)

In [14]:
for Name, Data in PicDatas.items():
    Data['MinIdx'] = min(Data['time'], key = Data['time'].get)

In [17]:
for Name, Data in PicDatas.items():
    print(f"Length: {Data['Len']}, MinIdx: {Data['MinIdx']}, MinTime: {Data['time'][Data['MinIdx']]}")

Length: 800.0, MinIdx: 13, MinTime: 534589
Length: 801.2015976020018, MinIdx: 14, MinTime: 464454
Length: 1468.6047800548656, MinIdx: 16, MinTime: 1587803
Length: 1538.1836691370768, MinIdx: 16, MinTime: 1730100
Length: 1539.2936691872671, MinIdx: 16, MinTime: 1640633
Length: 2063.9379835644286, MinIdx: 16, MinTime: 2828132
Length: 2063.9379835644286, MinIdx: 14, MinTime: 2735589
Length: 2202.9071700822983, MinIdx: 16, MinTime: 3019100
Length: 2307.552816296953, MinIdx: 13, MinTime: 3890009
Length: 2333.3829518533817, MinIdx: 15, MinTime: 3836745
Length: 2465.3245222485416, MinIdx: 15, MinTime: 4209810
Length: 2937.2095601097312, MinIdx: 15, MinTime: 5268709
Length: 3509.79087696119, MinIdx: 16, MinTime: 8007683
Length: 4405.814340164597, MinIdx: 16, MinTime: 11343134
Length: 6992.076158052056, MinIdx: 16, MinTime: 30591514
Length: 8811.628680329193, MinIdx: 16, MinTime: 47031003
